In [ ]:
import pandas as pd
import tiktoken
import openai

In [2]:
openai.api_key = ''

In [3]:
df = pd.read_csv('data_judul_berita.csv')

In [4]:
df.head()

,kategori,tanggal_publikasi,judul_berita,tokens
0,Kesra,2018-12-31,574 Pasangan Ikuti Itsbat dan Nikah Massal,28
1,Jakarta Hari Ini,2018-12-31,Sudinsos Jakut Kerahkan 68 P3S di Pusat Kerama...,41
2,Pemerintahan,2018-12-31,Anies Lantik Tiga Pejabat Pimpinan Tinggi Pratama,37
3,Jakarta Hari Ini,2018-12-31,Kelurahan Angke Terjunkan 60 Petugas PPSU di R...,44
4,Kesra,2018-12-31,Warga Marunda Himpun Bantuan Bagi Korban Tsunami,33


In [5]:
model_engine = "davinci"

In [6]:
df['context'] = df.kategori + "\n" + df.tanggal_publikasi + "\n\n" + df.judul_berita 
df.head()

,kategori,tanggal_publikasi,judul_berita,tokens,context
0,Kesra,2018-12-31,574 Pasangan Ikuti Itsbat dan Nikah Massal,28,Kesra\n2018-12-31\n\n574 Pasangan Ikuti Itsbat...
1,Jakarta Hari Ini,2018-12-31,Sudinsos Jakut Kerahkan 68 P3S di Pusat Kerama...,41,Jakarta Hari Ini\n2018-12-31\n\nSudinsos Jakut...
2,Pemerintahan,2018-12-31,Anies Lantik Tiga Pejabat Pimpinan Tinggi Pratama,37,Pemerintahan\n2018-12-31\n\nAnies Lantik Tiga ...
3,Jakarta Hari Ini,2018-12-31,Kelurahan Angke Terjunkan 60 Petugas PPSU di R...,44,Jakarta Hari Ini\n2018-12-31\n\nKelurahan Angk...
4,Kesra,2018-12-31,Warga Marunda Himpun Bantuan Bagi Korban Tsunami,33,Kesra\n2018-12-31\n\n Warga Marunda Himpun Ban...


In [8]:
df.drop('tokens', axis=1, inplace=True)

In [9]:
context = df['context']

In [10]:
context

0     Kesra\n2018-12-31\n\n574 Pasangan Ikuti Itsbat...
1     Jakarta Hari Ini\n2018-12-31\n\nSudinsos Jakut...
2     Pemerintahan\n2018-12-31\n\nAnies Lantik Tiga ...
3     Jakarta Hari Ini\n2018-12-31\n\nKelurahan Angk...
4     Kesra\n2018-12-31\n\n Warga Marunda Himpun Ban...
5     Jakarta Hari Ini\n2018-12-31\n\nSudinsos Jakti...
6     Wisata & Kuliner\n2018-12-31\n\nFestival Kulin...
7     Lintas Kota\n2018-12-31\n\nBupati Minta Aktivi...
8     Olahraga\n2018-12-31\n\nPemkot Jakut Apresiasi...
9     Lintas Kota\n2018-12-31\n\nSudah 60 Meter Kubi...
10    Jakarta Hari Ini\n2018-12-31\n\nBupati Tinjau ...
11    Jakarta Hari Ini\n2018-12-31\n\nRehab Total Ru...
12    Pemerintahan\n2018-12-31\n\nPemkot Jakut Dekla...
13    Jakarta Hari Ini\n2018-12-31\n\nSudin Gulkarma...
14    Jakarta Hari Ini\n2018-12-31\n\nPemkot Jaksel ...
15    Lintas Kota\n2018-12-31\n\nPBB Setu Babakan Di...
16    Jakarta Hari Ini\n2018-12-31\n\nPemkab Adakan ...
17    Jakarta Hari Ini\n2018-12-31\n\nPemprov DK

In [13]:
context

0     Kesra\n2018-12-31\n\n574 Pasangan Ikuti Itsbat...
1     Jakarta Hari Ini\n2018-12-31\n\nSudinsos Jakut...
2     Pemerintahan\n2018-12-31\n\nAnies Lantik Tiga ...
3     Jakarta Hari Ini\n2018-12-31\n\nKelurahan Angk...
4     Kesra\n2018-12-31\n\n Warga Marunda Himpun Ban...
5     Jakarta Hari Ini\n2018-12-31\n\nSudinsos Jakti...
6     Wisata & Kuliner\n2018-12-31\n\nFestival Kulin...
7     Lintas Kota\n2018-12-31\n\nBupati Minta Aktivi...
8     Olahraga\n2018-12-31\n\nPemkot Jakut Apresiasi...
9     Lintas Kota\n2018-12-31\n\nSudah 60 Meter Kubi...
10    Jakarta Hari Ini\n2018-12-31\n\nBupati Tinjau ...
11    Jakarta Hari Ini\n2018-12-31\n\nRehab Total Ru...
12    Pemerintahan\n2018-12-31\n\nPemkot Jakut Dekla...
13    Jakarta Hari Ini\n2018-12-31\n\nSudin Gulkarma...
14    Jakarta Hari Ini\n2018-12-31\n\nPemkot Jaksel ...
15    Lintas Kota\n2018-12-31\n\nPBB Setu Babakan Di...
16    Jakarta Hari Ini\n2018-12-31\n\nPemkab Adakan ...
17    Jakarta Hari Ini\n2018-12-31\n\nPemprov DK

In [14]:
import tiktoken #calculate tokens of each rows using tiktoken library

In [15]:
encoding = tiktoken.get_encoding("gpt2")

In [16]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [17]:
import numpy as np

tokens = []

for line in context:
    num_tokens = num_tokens_from_string(line,'gpt2')
    tokens.append(num_tokens)

arr_tokens = np.array(tokens)
toktoken = pd.DataFrame({"tokens" : arr_tokens})
toktoken.to_csv("tokens.csv", index= False)

print(arr_tokens)

[23 36 32 39 27 37 29 36 29 28 32 38 35 31 33 32 34 31 37 36 30 29 29 33
 35 31 29 27 38 32 35 37 33 34 33 28 34 36 29]


In [19]:
tokens= pd.DataFrame({"tokens" : arr_tokens})

In [20]:
tokens

,tokens
0,23
1,36
2,32
3,39
4,27
5,37
6,29
7,36
8,29
9,28


In [21]:
df['tokens'] = tokens['tokens']

In [22]:
df.drop('context', axis=1, inplace=True)

In [23]:
df

,kategori,tanggal_publikasi,judul_berita,tokens
0,Kesra,2018-12-31,574 Pasangan Ikuti Itsbat dan Nikah Massal,23
1,Jakarta Hari Ini,2018-12-31,Sudinsos Jakut Kerahkan 68 P3S di Pusat Kerama...,36
2,Pemerintahan,2018-12-31,Anies Lantik Tiga Pejabat Pimpinan Tinggi Pratama,32
3,Jakarta Hari Ini,2018-12-31,Kelurahan Angke Terjunkan 60 Petugas PPSU di R...,39
4,Kesra,2018-12-31,Warga Marunda Himpun Bantuan Bagi Korban Tsunami,27
5,Jakarta Hari Ini,2018-12-31,Sudinsos Jaktim Dirikan Tenda untuk Korban Keb...,37
6,Wisata & Kuliner,2018-12-31,Festival Kuliner Sambut Tahun Baru di Pulau La...,29
7,Lintas Kota,2018-12-31,Bupati Minta Aktivitas Menyelam Wisatawan Dida...,36
8,Olahraga,2018-12-31,Pemkot Jakut Apresiasi Atlet Berprestasi,29
9,Lintas Kota,2018-12-31,Sudah 60 Meter Kubik Material Vulkanik Dibersi...,28


In [25]:
df.to_csv('dataset_dg_token_terbaru.csv', index=False)